In [146]:
import os

CAR_PATH = os.path.join("datasets\cars", "car.txt")

In [147]:
import pandas as pd 
cars = pd.read_csv(CAR_PATH, delimiter = ",", header=None)
cars.columns = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety", "Acceptability"]
cars

,Buying,Maint,Doors,Persons,Lug_boot,Safety,Acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [148]:
for col in cars.columns:
    print(cars[col].value_counts())

low      432
high     432
med      432
vhigh    432
Name: Buying, dtype: int64
low      432
high     432
med      432
vhigh    432
Name: Maint, dtype: int64
3        432
4        432
5more    432
2        432
Name: Doors, dtype: int64
4       576
more    576
2       576
Name: Persons, dtype: int64
small    576
big      576
med      576
Name: Lug_boot, dtype: int64
low     576
high    576
med     576
Name: Safety, dtype: int64
unacc    1210
acc       384
good       69
vgood      65
Name: Acceptability, dtype: int64


In [149]:
# We have to stratify Acceptability since it's skewed
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(cars, cars[["Acceptability"]]): # Using StratifiedShuffleSplit and splitting using housing["income_cat"], we ensured that the proportion of each category of housing["income_cat"] is the same in the training and test sets
    strat_train_full_set = cars.loc[train_index]
    strat_test_set = cars.loc[test_index]
for train_index, valid_index in split.split(strat_train_full_set, strat_train_full_set[["Acceptability"]]):
    strat_train_set = strat_train_full_set.iloc[train_index] # loc does not work
    strat_valid_set = strat_train_full_set.iloc[valid_index]

In [150]:
strat_train_set["Acceptability"].value_counts() / len(strat_train_set)

unacc    0.700452
acc      0.221719
good     0.039819
vgood    0.038009
Name: Acceptability, dtype: float64

In [151]:
strat_valid_set["Acceptability"].value_counts() / len(strat_valid_set)

unacc    0.700361
acc      0.223827
good     0.039711
vgood    0.036101
Name: Acceptability, dtype: float64

In [152]:
strat_test_set["Acceptability"].value_counts() / len(strat_test_set)

unacc    0.699422
acc      0.222543
good     0.040462
vgood    0.037572
Name: Acceptability, dtype: float64

In [153]:
strat_train_set_data = strat_train_set.drop("Acceptability", axis=1) # drop labels for training set
strat_train_set_labels = strat_train_set["Acceptability"].copy()
strat_valid_set_data = strat_valid_set.drop("Acceptability", axis=1) # drop labels for training set
strat_valid_set_labels = strat_valid_set["Acceptability"].copy()

In [154]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

data_cat_attribs = ["Buying", "Maint", "Doors", "Persons", "Lug_boot", "Safety"]
data_pipeline = ColumnTransformer([
        ("oHCat", OneHotEncoder(), data_cat_attribs),
])
strat_train_set_data_encoded = data_pipeline.fit_transform(strat_train_set_data)
strat_valid_set_data_encoded = data_pipeline.fit_transform(strat_valid_set_data)

In [155]:
strat_train_set_data_encoded

<1105x21 sparse matrix of type '<class 'numpy.float64'>'
	with 6630 stored elements in Compressed Sparse Row format>

In [156]:
import numpy as np
strat_train_set_labels = np.array(strat_train_set_labels.values.tolist())
strat_train_set_labels = strat_train_set_labels.reshape(-1, 1) # Need to be a 1D matrix, not a 1D vector, for ordinal_encoder
strat_valid_set_labels = np.array(strat_valid_set_labels.values.tolist())
strat_valid_set_labels = strat_valid_set_labels.reshape(-1, 1)

In [157]:
strat_valid_set_labels.shape # Shape should be (277,1) not (277,)

(277, 1)

In [158]:
ordinal_encoder = OrdinalEncoder() # Instead of one-hot, we use ordinal. This is because the magnitude of the labels indicate some level difference
strat_train_set_labels_encoded = ordinal_encoder.fit_transform(strat_train_set_labels) # Change different text categories to numbers
strat_valid_set_labels_encoded = ordinal_encoder.fit_transform(strat_valid_set_labels)
strat_train_set_labels_encoded[:10]

array([[2.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [0.],
       [2.],
       [2.],
       [0.]])

In [159]:
print(np.unique(strat_train_set_labels_encoded))
print(np.unique(strat_valid_set_labels_encoded))
print(strat_train_set_labels_encoded.shape)
print(strat_valid_set_labels_encoded.shape)

[0. 1. 2. 3.]
[0. 1. 2. 3.]
(1105, 1)
(277, 1)


In [160]:
strat_train_set_labels_encoded = strat_train_set_labels_encoded.flatten() # Some algorithms expect a 1D vector not a 1D matrix
strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.flatten()

In [161]:
print(np.unique(strat_train_set_labels_encoded))
print(np.unique(strat_valid_set_labels_encoded))
print(strat_train_set_labels_encoded.shape)
print(strat_valid_set_labels_encoded.shape)

[0. 1. 2. 3.]
[0. 1. 2. 3.]
(1105,)
(277,)


In [162]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier(max_iter = 5000, random_state=42)
sgd_clf.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=5000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [165]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(sgd_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test indices such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8590785907859079
0.8590785907859079
0.8555858310626703


In [166]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = [
    # try 9 (3×3) combinations of hyperparameters
    {'n_estimators': [10,30,100], 'max_features': [8,10,12]},
    # then try 9 (9×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [10,30,100], 'max_features': [8,10,12]},
]
forest_clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(forest_clf, param_grid, cv=5, scoring='f1_weighted', return_train_score=True)
grid_search.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid=[{

In [167]:
for_clf = grid_search.best_estimator_
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(for_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.943089430894309
0.9349593495934959
0.9509536784741145


In [168]:
print(grid_search.best_params_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

{'max_features': 12, 'n_estimators': 30}
0.929691182772363 {'max_features': 8, 'n_estimators': 10}
0.9403809751274504 {'max_features': 8, 'n_estimators': 30}
0.9500815521965411 {'max_features': 8, 'n_estimators': 100}
0.9293743861022711 {'max_features': 10, 'n_estimators': 10}
0.9457516845255892 {'max_features': 10, 'n_estimators': 30}
0.9485008875115397 {'max_features': 10, 'n_estimators': 100}
0.9455154048776516 {'max_features': 12, 'n_estimators': 10}
0.9567003357810132 {'max_features': 12, 'n_estimators': 30}
0.9557733315721635 {'max_features': 12, 'n_estimators': 100}
0.9428576453627583 {'bootstrap': False, 'max_features': 8, 'n_estimators': 10}
0.9498782113510762 {'bootstrap': False, 'max_features': 8, 'n_estimators': 30}
0.9475272112434501 {'bootstrap': False, 'max_features': 8, 'n_estimators': 100}
0.945088272530604 {'bootstrap': False, 'max_features': 10, 'n_estimators': 10}
0.9512309058215088 {'bootstrap': False, 'max_features': 10, 'n_estimators': 30}
0.9557371218490074 {'bo

In [170]:
# Now let's try the above using randomizedsearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
}
forest_clf = RandomForestClassifier(random_state=42)
rnd_search = RandomizedSearchCV(forest_clf, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='f1_weighted', random_state=42)
rnd_search.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)
# The warnings are saying that some labels in y_true don't appear in y_pred

c:\users\taij\documents\calvin\myprojects\env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\taij\documents\calvin\myprojects\env\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [171]:
for_clf = rnd_search.best_estimator_
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(for_clf)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.9214092140921409
0.94579945799458
0.9427792915531336


In [172]:
print(rnd_search.best_params_)
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

{'max_features': 7, 'n_estimators': 48}
0.9422854273370739 {'max_features': 7, 'n_estimators': 180}
0.9312092226691626 {'max_features': 5, 'n_estimators': 15}
0.9203173415131247 {'max_features': 3, 'n_estimators': 72}
0.9269449296162904 {'max_features': 5, 'n_estimators': 21}
0.9423627571854498 {'max_features': 7, 'n_estimators': 122}
0.9242212728297717 {'max_features': 3, 'n_estimators': 75}
0.9217607183647717 {'max_features': 3, 'n_estimators': 88}
0.9453559493693303 {'max_features': 5, 'n_estimators': 100}
0.9206255817212562 {'max_features': 3, 'n_estimators': 150}
0.8300657246490976 {'max_features': 5, 'n_estimators': 2}
0.9284530728497802 {'max_features': 4, 'n_estimators': 158}
0.9421342448164106 {'max_features': 6, 'n_estimators': 130}
0.9243939933804599 {'max_features': 4, 'n_estimators': 21}
0.862881023184831 {'max_features': 1, 'n_estimators': 58}
0.9430050474629227 {'max_features': 6, 'n_estimators': 89}
0.8608553933929777 {'max_features': 1, 'n_estimators': 59}
0.9433849702

In [173]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded)
skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
clone_clf = clone(log_reg)
for train_index, test_index in skfolds.split(strat_train_set_data_encoded, strat_train_set_labels_encoded): # provide train and test index such that in each fold-validation, the class 5 & class not-5 proportions of the training and test sets are the same as the entire dataset.
    X_train_folds = strat_train_set_data_encoded[train_index]
    y_train_folds = strat_train_set_labels_encoded[train_index]
    X_test_fold = strat_train_set_data_encoded[test_index]
    y_test_fold = strat_train_set_labels_encoded[test_index]
    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.8753387533875339
0.8644986449864499
0.8664850136239782


In [226]:
# Usually we would keep on tuning the paramters on the validation set, here I picked the model that performs the best and use it on the validation set
for_clf = RandomForestClassifier(n_estimators=30, max_features=12, random_state=42)
for_clf.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded) # Fits the training set, but predicts the valid set
y_pred = for_clf.predict(strat_valid_set_data_encoded)

In [227]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
print(confusion_matrix(strat_valid_set_labels_encoded, y_pred)) # arguments: actual, predict

[[ 60   0   1   1]
 [  1   9   0   1]
 [  1   0 193   0]
 [  0   0   0  10]]


In [228]:
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97        62
         1.0       1.00      0.82      0.90        11
         2.0       0.99      0.99      0.99       194
         3.0       0.83      1.00      0.91        10

    accuracy                           0.98       277
   macro avg       0.95      0.95      0.94       277
weighted avg       0.98      0.98      0.98       277



In [229]:
print(precision_score(strat_valid_set_labels_encoded, y_pred, average='weighted'))
print(recall_score(strat_valid_set_labels_encoded, y_pred, average='weighted'))
print(f1_score(strat_valid_set_labels_encoded, y_pred, average='weighted')) # the greater the precison & recall, the greater the f1_score and the better the classifier!

0.9831528279181708
0.9819494584837545
0.9819166393173612


In [242]:
import tensorflow as tf
from tensorflow import keras
# GPUs config
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus) # Looks like we have one GPU, nice!
# Config for GPU memory growth if needed
tf.config.experimental.set_memory_growth(gpus[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [295]:
def build_model(n_hidden=3, n_neurons=30, learning_rate=3e-3, input_shape=[21]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(4, activation="softmax")) # activation="sigmoid" for binary outputs
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(metrics= ['acc'], loss="sparse_categorical_crossentropy", optimizer=optimizer)
    return model

In [296]:
keras_reg = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [297]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-5, 3e-1),
}

# strat_train_set_data_encoded = strat_train_set_data_encoded.toarray()
# strat_train_set_labels_encoded = strat_train_set_labels_encoded.toarray()
# strat_valid_set_data_encoded = strat_valid_set_data_encoded.toarray()
# strat_valid_set_labels_encoded = strat_valid_set_labels_encoded.toarray()

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=50, cv=3, verbose=2)
rnd_search_cv.fit(strat_train_set_data_encoded, strat_train_set_labels_encoded, epochs=100,
                  validation_data=(strat_valid_set_data_encoded, strat_valid_set_labels_encoded),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)])

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] learning_rate=0.020559064593747168, n_hidden=0, n_neurons=69 ....
Train on 736 samples, validate on 277 samples
Epoch 1/100
 32/736 [>.............................] - ETA: 2s - loss: 1.2998 - acc: 0.4062

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


736/736 [==============================] - 0s 305us/sample - loss: 1.1252 - acc: 0.5516 - val_loss: 1.0613 - val_acc: 0.6101
Epoch 2/100
736/736 [==============================] - 0s 80us/sample - loss: 0.9920 - acc: 0.6467 - val_loss: 0.9634 - val_acc: 0.6570
Epoch 3/100
736/736 [==============================] - 0s 76us/sample - loss: 0.9141 - acc: 0.6889 - val_loss: 0.9021 - val_acc: 0.6823
Epoch 4/100
736/736 [==============================] - 0s 76us/sample - loss: 0.8621 - acc: 0.7011 - val_loss: 0.8584 - val_acc: 0.6931
Epoch 5/100
736/736 [==============================] - 0s 73us/sample - loss: 0.8235 - acc: 0.7079 - val_loss: 0.8243 - val_acc: 0.6968
Epoch 6/100
736/736 [==============================] - 0s 76us/sample - loss: 0.7928 - acc: 0.7133 - val_loss: 0.7965 - val_acc: 0.7112
Epoch 7/100
736/736 [==============================] - 0s 73us/sample - loss: 0.7668 - acc: 0.7188 - val_loss: 0.7725 - val_acc: 0.7148
Epoch 8/100
736/736 [==============================] - 0s 7

Epoch 61/100
736/736 [==============================] - 0s 76us/sample - loss: 0.4353 - acc: 0.8478 - val_loss: 0.4638 - val_acc: 0.8231
Epoch 62/100
736/736 [==============================] - 0s 77us/sample - loss: 0.4333 - acc: 0.8492 - val_loss: 0.4620 - val_acc: 0.8267
Epoch 63/100
736/736 [==============================] - 0s 74us/sample - loss: 0.4314 - acc: 0.8478 - val_loss: 0.4602 - val_acc: 0.8267
Epoch 64/100
736/736 [==============================] - 0s 77us/sample - loss: 0.4293 - acc: 0.8478 - val_loss: 0.4584 - val_acc: 0.8303
Epoch 65/100
736/736 [==============================] - 0s 77us/sample - loss: 0.4274 - acc: 0.8478 - val_loss: 0.4566 - val_acc: 0.8339
Epoch 66/100
736/736 [==============================] - 0s 75us/sample - loss: 0.4256 - acc: 0.8505 - val_loss: 0.4549 - val_acc: 0.8339
Epoch 67/100
736/736 [==============================] - 0s 72us/sample - loss: 0.4237 - acc: 0.8505 - val_loss: 0.4533 - val_acc: 0.8339
Epoch 68/100
736/736 [===================

[CV]  learning_rate=0.020559064593747168, n_hidden=0, n_neurons=69, total=   5.9s
[CV] learning_rate=0.020559064593747168, n_hidden=0, n_neurons=69 ....
Train on 737 samples, validate on 277 samples
Epoch 1/100


...

[CV]  learning_rate=0.0005288675836532477, n_hidden=1, n_neurons=73, total=   7.2s
Train on 1105 samples, validate on 277 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 17.0min finished


1105/1105 [==============================] - 0s 310us/sample - loss: 0.8267 - acc: 0.7023 - val_loss: 0.6893 - val_acc: 0.7004
Epoch 2/100
1105/1105 [==============================] - 0s 93us/sample - loss: 0.4317 - acc: 0.8389 - val_loss: 0.3704 - val_acc: 0.8159
Epoch 3/100
1105/1105 [==============================] - 0s 89us/sample - loss: 0.3193 - acc: 0.8624 - val_loss: 0.3158 - val_acc: 0.8664
Epoch 4/100
1105/1105 [==============================] - 0s 88us/sample - loss: 0.2472 - acc: 0.8950 - val_loss: 0.2057 - val_acc: 0.9097
Epoch 5/100
1105/1105 [==============================] - 0s 84us/sample - loss: 0.2360 - acc: 0.9176 - val_loss: 0.2095 - val_acc: 0.9061
Epoch 6/100
1105/1105 [==============================] - 0s 82us/sample - loss: 0.1618 - acc: 0.9357 - val_loss: 0.3493 - val_acc: 0.8664
Epoch 7/100
1105/1105 [==============================] - 0s 86us/sample - loss: 0.1863 - acc: 0.9330 - val_loss: 0.1708 - val_acc: 0.9422
Epoch 8/100
1105/1105 [======================

1105/1105 [==============================] - 0s 84us/sample - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0167 - val_acc: 0.9964
Epoch 61/100
1105/1105 [==============================] - 0s 87us/sample - loss: 0.0010 - acc: 1.0000 - val_loss: 0.0168 - val_acc: 0.9964
Epoch 62/100
1105/1105 [==============================] - 0s 88us/sample - loss: 9.8174e-04 - acc: 1.0000 - val_loss: 0.0164 - val_acc: 0.9964
Epoch 63/100
1105/1105 [==============================] - 0s 86us/sample - loss: 9.6126e-04 - acc: 1.0000 - val_loss: 0.0154 - val_acc: 0.9964
Epoch 64/100
1105/1105 [==============================] - 0s 88us/sample - loss: 9.3061e-04 - acc: 1.0000 - val_loss: 0.0144 - val_acc: 0.9964
Epoch 65/100
1105/1105 [==============================] - 0s 84us/sample - loss: 8.9928e-04 - acc: 1.0000 - val_loss: 0.0149 - val_acc: 0.9964
Epoch 66/100
1105/1105 [==============================] - 0s 88us/sample - loss: 8.8270e-04 - acc: 1.0000 - val_loss: 0.0154 - val_acc: 0.9964
Epoch 67/100
1105/11

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000002702310DA08>,
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000027018043988>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score

In [298]:
rnd_search_cv.best_params_

{'learning_rate': 0.14177093519779227, 'n_hidden': 3, 'n_neurons': 51}

In [299]:
rnd_search_cv.best_score_ # score is the higher (more positive) the better

0.9610859759792483

In [300]:
cvres = rnd_search_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]): # mean score is the higher (more positive) the better, since this is sklearn's RandomizedSearchCV
    print(mean_score, params)

0.8307692343832681 {'learning_rate': 0.020559064593747168, 'n_hidden': 0, 'n_neurons': 69}
0.7004524907375353 {'learning_rate': 0.0011268753490968147, 'n_hidden': 3, 'n_neurons': 19}
0.8524886963054605 {'learning_rate': 0.03151968704690126, 'n_hidden': 0, 'n_neurons': 75}
0.7085972931590016 {'learning_rate': 0.0012462717224446028, 'n_hidden': 2, 'n_neurons': 60}
0.6995475075902982 {'learning_rate': 0.00030452717450011745, 'n_hidden': 2, 'n_neurons': 11}
0.8624434212214267 {'learning_rate': 0.004221727736971434, 'n_hidden': 3, 'n_neurons': 66}
0.8361991113127627 {'learning_rate': 0.020748347697774953, 'n_hidden': 0, 'n_neurons': 10}
0.9447963731321274 {'learning_rate': 0.2308343538839298, 'n_hidden': 1, 'n_neurons': 14}
0.8941176279637608 {'learning_rate': 0.017580724755985492, 'n_hidden': 1, 'n_neurons': 43}
0.6479637865027691 {'learning_rate': 0.0005207535715729814, 'n_hidden': 0, 'n_neurons': 27}
0.3085972827214461 {'learning_rate': 0.00010987619052464836, 'n_hidden': 0, 'n_neurons':

In [301]:
print(rnd_search_cv.best_estimator_)
ANNmodel = rnd_search_cv.best_estimator_.model

In [325]:
y_pred = ANNmodel.predict_classes(strat_valid_set_data_encoded)

In [326]:
print(confusion_matrix(strat_valid_set_labels_encoded, y_pred)) # arguments: actual, predict

[[ 61   0   1   0]
 [  0  11   0   0]
 [  0   0 194   0]
 [  0   0   0  10]]


In [327]:
print(classification_report(strat_valid_set_labels_encoded, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      0.98      0.99        62
         1.0       1.00      1.00      1.00        11
         2.0       0.99      1.00      1.00       194
         3.0       1.00      1.00      1.00        10

    accuracy                           1.00       277
   macro avg       1.00      1.00      1.00       277
weighted avg       1.00      1.00      1.00       277



In [280]:
strat_train_full_set_data = strat_train_full_set.drop("Acceptability", axis=1)
strat_train_full_set_labels = strat_train_full_set["Acceptability"].copy()
strat_train_full_set_data_encoded = data_pipeline.fit_transform(strat_train_full_set_data)
strat_train_full_set_data_encoded = strat_train_full_set_data_encoded.toarray()
strat_train_full_set_labels = np.array(strat_train_full_set_labels.values.tolist())
strat_train_full_set_labels = strat_train_full_set_labels.reshape(-1, 1)
strat_train_full_set_labels_encoded = ordinal_encoder.fit_transform(strat_train_full_set_labels)
strat_train_full_set_labels_encoded = strat_train_full_set_labels_encoded.flatten()

Train on 1382 samples
Epoch 1/100
1382/1382 [==============================] - 0s 82us/sample - loss: 0.0385 - acc: 0.9957
Epoch 2/100
1382/1382 [==============================] - 0s 68us/sample - loss: 0.0176 - acc: 0.9993
Epoch 3/100
1382/1382 [==============================] - 0s 66us/sample - loss: 0.0104 - acc: 0.9993
Epoch 4/100
1382/1382 [==============================] - 0s 69us/sample - loss: 0.0078 - acc: 1.0000
Epoch 5/100
1382/1382 [==============================] - 0s 66us/sample - loss: 0.0058 - acc: 1.0000
Epoch 6/100
1382/1382 [==============================] - 0s 66us/sample - loss: 0.0047 - acc: 1.0000
Epoch 7/100
1382/1382 [==============================] - 0s 66us/sample - loss: 0.0037 - acc: 1.0000
Epoch 8/100
1382/1382 [==============================] - 0s 66us/sample - loss: 0.0034 - acc: 1.0000
Epoch 9/100
1382/1382 [==============================] - 0s 64us/sample - loss: 0.0030 - acc: 1.0000
Epoch 10/100
1382/1382 [==============================] - 0s 69us/sam

1382/1382 [==============================] - 0s 68us/sample - loss: 8.4800e-04 - acc: 1.0000
Epoch 28/100
1382/1382 [==============================] - 0s 67us/sample - loss: 8.1239e-04 - acc: 1.0000
Epoch 29/100
1382/1382 [==============================] - 0s 67us/sample - loss: 7.8274e-04 - acc: 1.0000
Epoch 30/100
1382/1382 [==============================] - 0s 70us/sample - loss: 7.4793e-04 - acc: 1.0000
Epoch 31/100
1382/1382 [==============================] - 0s 66us/sample - loss: 7.2474e-04 - acc: 1.0000
Epoch 32/100
1382/1382 [==============================] - 0s 68us/sample - loss: 6.9775e-04 - acc: 1.0000
Epoch 33/100
1382/1382 [==============================] - 0s 68us/sample - loss: 6.7766e-04 - acc: 1.0000
Epoch 34/100
1382/1382 [==============================] - 0s 66us/sample - loss: 6.5500e-04 - acc: 1.0000
Epoch 35/100
1382/1382 [==============================] - 0s 66us/sample - loss: 6.3754e-04 - acc: 1.0000
Epoch 36/100
1382/1382 [==============================] - 0

Epoch 54/100
1382/1382 [==============================] - 0s 67us/sample - loss: 3.8655e-04 - acc: 1.0000
Epoch 55/100
1382/1382 [==============================] - 0s 69us/sample - loss: 3.7944e-04 - acc: 1.0000
Epoch 56/100
1382/1382 [==============================] - 0s 65us/sample - loss: 3.6854e-04 - acc: 1.0000
Epoch 57/100
1382/1382 [==============================] - 0s 66us/sample - loss: 3.5976e-04 - acc: 1.0000
Epoch 58/100
1382/1382 [==============================] - 0s 67us/sample - loss: 3.5256e-04 - acc: 1.0000
Epoch 59/100
1382/1382 [==============================] - 0s 66us/sample - loss: 3.4663e-04 - acc: 1.0000
Epoch 60/100
1382/1382 [==============================] - 0s 67us/sample - loss: 3.4107e-04 - acc: 1.0000
Epoch 61/100
1382/1382 [==============================] - 0s 69us/sample - loss: 3.3303e-04 - acc: 1.0000
Epoch 62/100
1382/1382 [==============================] - 0s 69us/sample - loss: 3.2755e-04 - acc: 1.0000
Epoch 63/100
1382/1382 [======================

In [331]:
X_test_prepared

<346x21 sparse matrix of type '<class 'numpy.float64'>'
	with 2076 stored elements in Compressed Sparse Row format>

In [328]:
final_model = ANNmodel
X_test = strat_test_set.drop("Acceptability", axis=1) 
X_test_prepared = data_pipeline.transform(X_test) # we do not fit (only transform) here because we want to use the fitted values of the training set to predict this, i.e., use the training set's median values and extra attributes to transform the test set
X_test_prepared = X_test_prepared.toarray()
y_test = ordinal_encoder.transform(y_test)
y_test = y_test.flatten()
final_predictions = final_model.predict_classes(X_test_prepared)

In [329]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
print(precision_score(y_test, final_predictions, average='weighted'))
print(recall_score(y_test, final_predictions, average='weighted'))
print(f1_score(y_test, final_predictions, average='weighted'))
print(confusion_matrix(y_test, final_predictions))

1.0
1.0
1.0
[[ 77   0   0   0]
 [  0  14   0   0]
 [  0   0 242   0]
 [  0   0   0  13]]


In [330]:
y_pred = final_model.predict_classes(X_test_prepared)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        77
         1.0       1.00      1.00      1.00        14
         2.0       1.00      1.00      1.00       242
         3.0       1.00      1.00      1.00        13

    accuracy                           1.00       346
   macro avg       1.00      1.00      1.00       346
weighted avg       1.00      1.00      1.00       346

